In [1]:
import os

In [2]:
%pwd

'C:\\Users\\peguy\\PycharmProjects\\kidneydiseaseklassification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd


'C:\\Users\\peguy\\PycharmProjects\\kidneydiseaseklassification'

In [6]:
import dagshub
dagshub.init(repo_owner='Rustyfull', repo_name='kidney-disease-classification', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

C:\Users\peguy\anaconda3\envs\cancer\lib\site-packages\rich\live.py:260: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=bcff5de0-3546-45db-b9dd-263de3afe423&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=5059e44b96ca2f3866535e449947b0977814002e7b88599cda2983d83950ef44




Accessing as Rustyfull

Initialized MLflow to track repo "Rustyfull/kidney-disease-classification"

Repository Rustyfull/kidney-disease-classification initialized!

In [7]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [11]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/data"),
            mlflow_uri="https://dagshub.com/Rustyfull/kidney-disease-classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE


        )
        return eval_config

In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import  urlparse

In [16]:
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
            validation_split = 0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )


        self.valid_generator = valid_generator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()


    def save_score(self):
        scores = {"loss":self.score[0], "accuracy":self.score[1]}
        save_json(path=Path("scores.json"),data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss":self.score[0],"accuracy":self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#apiworkflow
                mlflow.keras.log_model(self.model,"model",registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model,"model")



In [18]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2026-02-17 10:10:46,012]: INFO: common: Yaml file: config\config.yml loaded successfully
[2026-02-17 10:10:46,016]: INFO: common: Yaml file: params.yml loaded successfully
[2026-02-17 10:10:46,018]: INFO: common: Created directory at: artifacts
Found 1471 images belonging to 2 classes.
92/92 [==============================] - 72s 784ms/step - loss: 153.4398 - accuracy: 0.8355


2026/02/17 10:12:00 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2026-02-17 10:12:01,677]: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.
INFO:tensorflow:Assets written to: C:\Users\peguy\AppData\Local\Temp\tmp_2mbe65r\model\data\model\assets
[2026-02-17 10:12:02,505]: INFO: builder_impl: Assets written to: C:\Users\peguy\AppData\Local\Temp\tmp_2mbe65r\model\data\model\assets


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2026/02/17 10:13:21 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
